### DATA SCRAPING

Ce fichier sera le support d'extraction de donnees concernant le diabete a partir du site https://www.diabete.qc.ca/. Ces donnees seront ensuite utilise dans l'implementation d'un modele d'apprentissage NLP avec le framework python : Rasa. L'objectif est d'obtenir la data necessaire a l'entrainement d'un modele sous forme de chatbot capable de fournir des informations sur le diabete afin de faire la sensibilisation sur cette maladie qui ne cesse de progresser.

### Implementation de l'algorithme de scraping

In [1]:
# importation des differents modules et classes de base
from selectolax.parser import HTMLParser
from loguru import logger
import sys 
import requests
import csv

In [2]:
# Mis en ploce des loggers
logger.remove()
logger.add("diabete.log", rotation="1000kb", level="WARNING")
logger.add(sys.stderr, level="INFO")

2

In [3]:
# recuperation de la page a scrapper
url = "https://www.diabete.qc.ca/le-diabete-en-questions/"
response = requests.get(url)
tree = HTMLParser(response.text)

In [4]:
def ul_extraction(ul):
    ul_resp = ''
    for li in list(ul.iter()):
        ul_resp += '-' + li.child.html +'\n'

    return ul_resp

In [5]:

def extract_response(balise):
    to_return = ''
    
    # recuperation du titre qui fera office d'intent pour le model
    title = balise.css_first("span.accordion__header__title").child.html
        
    # recuperation des differentes du texte faisant office de reponse
    resps = list(balise.css_first("div.accordion__sub-rows").iter())
    
    for resp in resps:
        if resp.tag == 'p':
            to_return += resp.child.html + '\n'
        elif resp.tag == 'ul':
            to_return += ul_extraction(resp) + '\n'
            
    return {'title' : title, 
            'description' : to_return}

In [6]:
def fonction(container):
    liste = []
    accordions = container.css("section.accordion")
    for accordion in accordions:
        liste.append(extract_response(accordion))
    return liste

In [7]:
sections = tree.css_first("section.block-accordions-group.theme-block.aligndefault")
informations = fonction(sections)

print(informations[1])

{'title': 'Quelles sont les complications du diabète ?', 'description': 'Les complications liées au diabète ont une origine commune : l’excédent de glucose dans le sang. Les complications du diabète peuvent modifier grandement la vie d’une personne diabétique. Un bon contrôle des glycémies permet de retarder et parfois même de limiter l’apparition des complications.\nÀ la longue, un taux de sucre élevé dans le sang entraîne certaines complications, notamment au niveau des yeux, des reins, des nerfs, du cœur et des vaisseaux sanguins.&nbsp;\n'}


In [8]:
with open("diabetes_data.csv", "w", encoding="utf-8", newline= "") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["title", "description"])
        writer.writeheader()
        writer.writerows(informations)